### **<u>main.py</u>**

The following Python code uses the PyQt6 library to create a GUI application named "MedTool+". The application has a sidebar and a main area. The sidebar contains several buttons including "Home", "Reminders", "Manage DB", "Logs", and "User Settings". Each button, when clicked, changes the content of the main area to a corresponding page. The pages are implemented in separate Python modules and are added to a QStackedWidget in the main area. The QStackedWidget allows switching between different widgets (pages) based on the button clicked in the sidebar.

The "hamburger" button in the sidebar toggles the visibility of the other buttons. The MainWindow class connects each button's clicked signal to a lambda function that sets the current widget of the QStackedWidget to the corresponding page. The application starts by creating an instance of QApplication, an instance of MainWindow, showing the window, and starting the application's event loop.

In [ ]:
from PyQt6.QtWidgets import QApplication, QMainWindow, QStackedWidget, QVBoxLayout, QWidget, QPushButton, QFrame, QHBoxLayout
from PyQt6 import QtCore
from qtawesome import icon
from PyQt6.QtWidgets import QButtonGroup, QLabel, QLineEdit, QPushButton
from PyQt6.QtGui import QIcon, QAction

from pages.user_settings import UserSettingsPage
from pages.logs import LogsPage
from pages.managedb import ManagedbPage
from pages.reminders import RemindersPage
from pages.home import HomePage

class SideBar(QWidget):
    def __init__(self):
        super().__init__()
        self.layout = QVBoxLayout()
        self.setFixedWidth(150)
        self.setStyleSheet("background-color: lightgrey;")  # Set the color of the sidebar
        self.buttons = {
            "hamburger": QPushButton(icon('fa5s.bars'), ""),  # Add the hamburger button here
            "home": QPushButton(icon('fa5s.home'), "Home"),
            "reminders": QPushButton(icon('fa5s.bell'), "Reminders"),
            "managedb": QPushButton(icon('fa5s.database'), "Manage DB"),
            "logs": QPushButton(icon('fa5s.book'), "Logs"),
            "settings": QPushButton(icon('fa5s.cog'), "User Settings"),
        }
        self.buttons["hamburger"].setFixedWidth(22)  # Set the width of the hamburger button
        self.button_group = QButtonGroup()  # Create a button group
        for btn in self.buttons.values():
            btn.setCheckable(True)
            btn.setIconSize(QtCore.QSize(22, 22))
            btn.setStyleSheet('''
                QPushButton {
                    text-align: left;
                    background-color: lightgrey;
                }
                QPushButton:checked {
                    background-color: lightblue;
                }
            ''')  # Set the style of the button
            self.layout.addWidget(btn)
            self.button_group.addButton(btn)
        self.setLayout(self.layout)
        self.buttons["home"].setChecked(True)  # Set the "Home" button as checked

class MainWindow(QMainWindow):
    def __init__(self, width=725, height=300):
        super().__init__()
        self.setWindowTitle("MedTool+")
        self.setWindowIcon(QIcon("assets/img/logo.png"))  # Set the application icon
        self.resize(width, height)  # Set the initial size of the window
        self.main_widget = QStackedWidget()
        self.sidebar = SideBar()
        self.home_page = HomePage()  # Create the home page
        self.reminders_page = RemindersPage()  # Create the reminders page
        self.managedb_page = ManagedbPage()  # Create the manage database page
        self.logs_page = LogsPage()  # Create the logs page
        self.user_settings_page = UserSettingsPage()  # Create the user settings page
        self.main_widget.addWidget(self.home_page)
        self.main_widget.addWidget(self.reminders_page)
        self.main_widget.addWidget(self.managedb_page)
        self.main_widget.addWidget(self.logs_page)
        self.main_widget.addWidget(self.user_settings_page)
        self.sidebar.buttons["settings"].clicked.connect(lambda: self.main_widget.setCurrentIndex(self.main_widget.count() - 1))
        self.sidebar.buttons["hamburger"].clicked.connect(self.toggle_sidebar)  # Connect the hamburger button to the toggle_sidebar method
        self.central_widget = QWidget()  # Create a central widget
        self.layout = QHBoxLayout(self.central_widget)  # Set the layout of the central widget
        self.layout.addWidget(self.sidebar)
        self.layout.addWidget(self.main_widget)
        self.setCentralWidget(self.central_widget)  # Set the central widget of the main window
        self.connect_signals()
        self.main_widget.setCurrentIndex(0)  # Set the current index to 0

    def toggle_sidebar(self):
        for btn in self.sidebar.buttons.values():
            if btn.text() != "":  # Exclude the hamburger button
                btn.setVisible(not btn.isVisible())  # Toggle the visibility of the button

    def connect_signals(self):
        self.sidebar.buttons["home"].clicked.connect(lambda: self.main_widget.setCurrentIndex(0))
        self.sidebar.buttons["reminders"].clicked.connect(lambda: self.main_widget.setCurrentIndex(1))
        self.sidebar.buttons["managedb"].clicked.connect(lambda: self.main_widget.setCurrentIndex(2))
        self.sidebar.buttons["logs"].clicked.connect(lambda: self.main_widget.setCurrentIndex(3))
        self.sidebar.buttons["settings"].clicked.connect(lambda: self.main_widget.setCurrentIndex(4))
    

if __name__ == "__main__":
    app = QApplication([])
    window = MainWindow()
    window.show()
    app.exec()

### **<u>home.py</u>**

The following Python code defines a HomePage class for a PyQt6 GUI application. The HomePage class inherits from QWidget and represents the home page of the application.

The home page contains three forms for adding users, medications, and reminders to a SQLite database. Each form has a set of input fields and a button. When the button is clicked, the corresponding add_ method is called, which inserts the input data into the database.

The user form allows adding a user with a type (Child or Admin), name, and age. The medication form allows adding a medication with a name, minimum interval, maximum dose, and age appropriateness. The reminder form allows adding a reminder with a person ID, medication ID, reminder time, and active status.

There is also a refresh button that attempts to refresh the database connection (a refresh_database method attempts to refresh the database connection and displays a confirmation message if successful, or an error message if an OperationalError occurs).

A QTimer object is used to update the reminder time every minute.

The location of the SQLite database is read from a config.json file.

The add_ methods use the sqlite3 library to connect to the SQLite database and execute SQL INSERT statements. After inserting the data, a confirmation message is displayed using a QMessageBox. The add_reminder method also logs the creation of the reminder in the database.

In [ ]:
from PyQt6.QtCore import QTimer, QDateTime, Qt
from PyQt6.QtWidgets import QLabel, QVBoxLayout, QWidget, QPushButton, QFormLayout, QLineEdit, QComboBox, QStackedLayout, QSpinBox, QDoubleSpinBox, QDateTimeEdit, QCheckBox, QMessageBox
import sqlite3
import os
import json

class HomePage(QWidget):
    def __init__(self):
        super().__init__()

        self.layout = QVBoxLayout()

        # User form
        self.user_form = QFormLayout()
        self.user_type = QComboBox()
        self.user_type.addItems(["Child", "Admin"])
        self.user_name = QLineEdit()
        self.user_age = QSpinBox()
        self.user_form.addRow("Type:", self.user_type)
        self.user_form.addRow("Name:", self.user_name)
        self.user_form.addRow("Age:", self.user_age)
        self.add_user_button = QPushButton("Add User")
        self.add_user_button.clicked.connect(self.add_user)
        self.layout.addLayout(self.user_form)
        self.layout.addWidget(self.add_user_button)

        # Medication form
        self.med_form = QFormLayout()
        self.med_name = QLineEdit()
        self.med_min_interval = QSpinBox()
        self.med_max_dose = QSpinBox()
        self.med_age_appropriate = QLineEdit()
        self.med_form.addRow("Name:", self.med_name)
        self.med_form.addRow("Min Interval:", self.med_min_interval)
        self.med_form.addRow("Max Dose:", self.med_max_dose)
        self.med_form.addRow("Age Appropriate:", self.med_age_appropriate)
        self.add_med_button = QPushButton("Add Medication")
        self.add_med_button.clicked.connect(self.add_medication)
        self.layout.addLayout(self.med_form)
        self.layout.addWidget(self.add_med_button)

        # Reminder form
        self.reminder_form = QFormLayout()
        self.reminder_person_id = QSpinBox()
        self.reminder_med_id = QSpinBox()
        self.reminder_time = QDateTimeEdit()
        self.reminder_time.setDateTime(QDateTime.currentDateTime())
        self.reminder_active = QCheckBox()
        self.reminder_form.addRow("Person ID:", self.reminder_person_id)
        self.reminder_form.addRow("Medication ID:", self.reminder_med_id)
        self.reminder_form.addRow("Reminder Time:", self.reminder_time)
        self.reminder_form.addRow("Active:", self.reminder_active)
        self.add_reminder_button = QPushButton("Add Reminder")
        self.add_reminder_button.clicked.connect(self.add_reminder)
        self.layout.addLayout(self.reminder_form)
        self.layout.addWidget(self.add_reminder_button)

        # Refresh button
        self.refresh_button = QPushButton("Refresh Database")
        self.refresh_button.clicked.connect(self.refresh_database)
        self.layout.addWidget(self.refresh_button)

        self.setLayout(self.layout)

        # Timer to update reminder_time every minute
        self.timer = QTimer()
        self.timer.timeout.connect(self.update_reminder_time)
        self.timer.start(60000)  # 60000 milliseconds = 1 minute

        # Get database location from config.json
        with open(os.path.join(os.path.dirname(os.path.dirname(__file__)), 'config.json')) as f:
            self.db_location = json.load(f)['databaseLocation']

    def add_user(self):
        conn = sqlite3.connect(self.db_location)
        cursor = conn.cursor()
        cursor.execute("""
            INSERT INTO people (name, age, type)
            VALUES (?, ?, ?)
        """, (self.user_name.text(), self.user_age.value(), self.user_type.currentText()))
        conn.commit()
        conn.close()

        # Confirmation message
        QMessageBox.information(self, "User Added", f"[{self.user_type.currentText()}] {self.user_name.text()} has been added to the database.")


    def add_medication(self):
        conn = sqlite3.connect(self.db_location)
        cursor = conn.cursor()
        cursor.execute("""
            INSERT INTO medications (name, min_interval, max_dose_per_day, age_appropriate)
            VALUES (?, ?, ?, ?)
        """, (self.med_name.text(), self.med_min_interval.value(), self.med_max_dose.value(), self.med_age_appropriate.text()))
        conn.commit()
        conn.close()

        # Confirmation message
        QMessageBox.information(self, "Medication Added", f"{self.med_name.text()} has been added to the database.")

    def add_log(self):
        conn = sqlite3.connect(self.db_location)
        cursor = conn.cursor()
        cursor.execute("""
            INSERT INTO log (person_id, medication_id, added_time)
            VALUES (?, ?, ?)
        """, (self.log_person_id.value(), self.log_med_id.value(), self.log_added_time.dateTime().toString(Qt.ISODate)))
        conn.commit()
        conn.close()

    def add_reminder(self):
        conn = sqlite3.connect(self.db_location)
        cursor = conn.cursor()
        cursor.execute("""
            INSERT INTO reminders (person_id, medication_id, reminder_time, active)
            VALUES (?, ?, ?, ?)
        """, (self.reminder_person_id.value(), self.reminder_med_id.value(), self.reminder_time.dateTime().toString(Qt.ISODate), self.reminder_active.isChecked()))
        reminder_id = cursor.lastrowid  # Get the ID of the newly inserted reminder

        # Log the creation of the reminder
        cursor.execute("""
            INSERT INTO log (reminder_id, action, timestamp)
            VALUES (?, ?, ?)
        """, (reminder_id, "Reminder created", QDateTime.currentDateTime().toString(Qt.ISODate)))

        conn.commit()
        conn.close()

        # Confirmation message
        QMessageBox.information(self, "Reminder Added", f"Reminder for person {self.reminder_person_id.value()} for medication {self.reminder_med_id.value()} has been added to the database.")

    def update_reminder_time(self):
        self.reminder_time.setDateTime(QDateTime.currentDateTime())

    def refresh_database(self):
        try:
            conn = sqlite3.connect(self.db_location)
            conn.close()

            # Confirmation message
            QMessageBox.information(self, "Refresh Database", "Database has been refreshed.")
        except sqlite3.OperationalError:
            # Error message
            QMessageBox.critical(self, "Refresh Database", "Failed to refresh the database. Please check the database file.")

### **<u>reminders.py</u>**

The following Python code defines a RemindersPage class for a PyQt6 GUI application. The RemindersPage class inherits from QWidget and represents the reminders page of the application.

The reminders page contains a table that displays active reminders from a SQLite database. Each row in the table represents a reminder and includes the ID, person's name, medication name, reminder time, and a countdown to the reminder time.

The table is updated every second by a QTimer object. The update_countdown method calculates the time remaining until each reminder and updates the countdown column of the table. If the reminder time has passed, it sets the countdown to "Time's up!".

There is also a refresh button that reloads the data from the database when clicked. The load_data method connects to the SQLite database, executes a SQL SELECT statement to fetch the active reminders, and populates the table with the fetched data.

The location of the SQLite database is read from a config.json file.

In [ ]:
from PyQt6.QtWidgets import QLabel, QVBoxLayout, QWidget, QTableWidget, QTableWidgetItem, QPushButton
from PyQt6.QtCore import QTimer, QDateTime, Qt
import sqlite3
import os
import json

class RemindersPage(QWidget):
    def __init__(self):
        super().__init__()
        with open(os.path.join(os.path.dirname(os.path.dirname(__file__)), 'config.json')) as f:
            self.db_location = json.load(f)['databaseLocation']
        self.layout = QVBoxLayout()

        self.table = QTableWidget()
        self.layout.addWidget(self.table)

        self.refresh_button = QPushButton("Refresh")
        self.refresh_button.clicked.connect(self.load_data)
        self.layout.addWidget(self.refresh_button)

        self.setLayout(self.layout)

        self.timer = QTimer()
        self.timer.timeout.connect(self.update_countdown)
        self.timer.start(1000)  # Update every second

        self.load_data()

    def load_data(self):
        conn = sqlite3.connect(self.db_location)
        cursor = conn.cursor()

        # Check if the reminders table exists
        cursor.execute("""
            SELECT name FROM sqlite_master WHERE type='table' AND name='reminders';
        """)
        if cursor.fetchone() is None:
            # The reminders table does not exist, so return without loading data
            conn.close()
            return

        cursor.execute("""
            SELECT r.id, p.name, m.name, r.reminder_time, r.active
            FROM reminders r
            JOIN people p ON r.person_id = p.id
            JOIN medications m ON r.medication_id = m.id
            WHERE r.active = 1
        """)
        reminders = cursor.fetchall()

        self.table.setRowCount(len(reminders))
        self.table.setColumnCount(5)
        self.table.setHorizontalHeaderLabels(["ID", "Person", "Medication", "Reminder Time", "Countdown"])

        for i, (id, person, medication, reminder_time, active) in enumerate(reminders):
            self.table.setItem(i, 0, QTableWidgetItem(str(id)))
            self.table.setItem(i, 1, QTableWidgetItem(person))
            self.table.setItem(i, 2, QTableWidgetItem(medication))
            self.table.setItem(i, 3, QTableWidgetItem(reminder_time))
            self.table.setItem(i, 4, QTableWidgetItem(""))  # Countdown will be updated by update_countdown

        conn.close()

    def update_countdown(self):
        current_time = QDateTime.currentDateTime()

        for i in range(self.table.rowCount()):
            reminder_time = QDateTime.fromString(self.table.item(i, 3).text(), Qt.ISODate)

            if reminder_time > current_time:
                countdown = current_time.secsTo(reminder_time)
                hours, remainder = divmod(countdown, 3600)
                minutes, seconds = divmod(remainder, 60)
                self.table.setItem(i, 4, QTableWidgetItem(f"{hours:02}:{minutes:02}:{seconds:02}"))
            else:
                self.table.setItem(i, 4, QTableWidgetItem("Time's up!"))

### **<u>manage_db.py</u>**

The following Python code defines a ManagedbPage class for a PyQt6 GUI application. The ManagedbPage class inherits from QWidget and represents a page for managing a SQLite database.

The page contains several buttons for managing the database: "Create DB", "Delete DB", "Create Backup", and "Rename DB". Each button is connected to a corresponding method that performs the action.

The create_db method creates a new SQLite database with tables for people, medications, logs, and reminders. If a database already exists at the specified location, it displays a warning message.

The delete_db method deletes the existing database after confirming the database name with the user. If the entered name matches the database name, it deletes the database and updates the database location in the config.json file.

The create_backup method creates a backup of the existing database by copying the database file to a new file with "-backup" appended to the name.

The rename_db method renames the existing database to a new name entered by the user. It updates the database location in the config.json file to reflect the new name.

The location of the SQLite database is read from and written to a config.json file.

In [ ]:
import os
import json
from PyQt6.QtWidgets import QVBoxLayout, QWidget, QLabel, QPushButton, QLineEdit, QHBoxLayout, QMessageBox, QInputDialog
from PyQt6.QtCore import Qt
from shutil import copyfile

import sqlite3

class ManagedbPage(QWidget):
    def __init__(self):
        super().__init__()
        self.layout = QVBoxLayout()

        # Create DB button
        self.create_db_button = QPushButton("Create DB")
        self.create_db_button.clicked.connect(self.create_db)
        self.layout.addWidget(self.create_db_button)

        # Delete DB button
        self.delete_db_button = QPushButton("Delete DB")
        self.delete_db_button.clicked.connect(self.delete_db)
        self.layout.addWidget(self.delete_db_button)

        # Create backup button
        self.create_backup_button = QPushButton("Create Backup")
        self.create_backup_button.clicked.connect(self.create_backup)
        self.layout.addWidget(self.create_backup_button)

        # Rename DB section
        self.rename_db_layout = QHBoxLayout()
        self.rename_db_input = QLineEdit()
        self.rename_db_button = QPushButton("Rename DB")
        self.rename_db_button.clicked.connect(self.rename_db)
        self.rename_db_layout.addWidget(self.rename_db_input)
        self.rename_db_layout.addWidget(self.rename_db_button)
        self.layout.addLayout(self.rename_db_layout)

        # Other useful buttons
        # Add other buttons as needed

        self.setLayout(self.layout)

    def create_db(self):
        config_path = os.path.join(os.path.dirname(os.path.dirname(__file__)), 'config.json')
        with open(config_path, 'r+') as f:
            data = json.load(f)
            db_path = data['databaseLocation']
            if not db_path:  # If the JSON location is empty
                db_path = os.path.join(os.path.dirname(os.path.dirname(__file__)), 'new_database.db')
                data['databaseLocation'] = db_path
            conn = sqlite3.connect(db_path)
            cursor = conn.cursor()

            # Check if any tables exist in the database
            cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
            tables = cursor.fetchall()

            if not tables:  # If no tables exist, the database is considered empty
                # Create people table
                cursor.execute("""
                    CREATE TABLE people (
                        id INTEGER PRIMARY KEY,
                        name TEXT,
                        age INTEGER,
                        type TEXT
                    )
                """)

                # Create medications table
                cursor.execute("""
                    CREATE TABLE medications (
                        id INTEGER PRIMARY KEY,
                        name TEXT,
                        min_interval INTEGER,
                        max_dose_per_day INTEGER,
                        age_appropriate TEXT,
                        quantity INTEGER,
                        price REAL
                    )
                """)

                # Create log table
                cursor.execute("""
                    CREATE TABLE log (
                        id INTEGER PRIMARY KEY,
                        reminder_id INTEGER,
                        action TEXT,
                        timestamp TEXT
                    )
                """)

                # Create reminders table
                cursor.execute("""
                    CREATE TABLE reminders (
                        id INTEGER PRIMARY KEY,
                        person_id INTEGER,
                        medication_id INTEGER,
                        reminder_time TEXT,
                        active INTEGER,
                        FOREIGN KEY(person_id) REFERENCES people(id),
                        FOREIGN KEY(medication_id) REFERENCES medications(id)
                    )
                """)

                QMessageBox.information(self, "Success", "Database successfully created.")
                print(f"Database successfully created at {db_path}.")
            else:
                QMessageBox.warning(self, "Failed", "Database already exists and is not empty.")

            conn.close()
            f.seek(0)
            json.dump(data, f, indent=4)
            f.truncate()

    def delete_db(self):
        config_path = os.path.join(os.path.dirname(os.path.dirname(__file__)), 'config.json')
        with open(config_path, 'r+') as f:
            data = json.load(f)
            if os.path.exists(data['databaseLocation']):
                db_name = os.path.basename(data['databaseLocation'])
                confirm_name, ok = QInputDialog.getText(self, "Confirm Delete", f"Please re-enter the name of this db {db_name} in the textbox below:")
                if ok and confirm_name == db_name:
                    os.remove(data['databaseLocation'])
                    data['databaseLocation'] = ""
                    f.seek(0)
                    json.dump(data, f, indent=4)
                    f.truncate()
                    QMessageBox.information(self, "Success", "Database successfully deleted.")
                elif ok:
                    QMessageBox.warning(self, "Failed", "Database name does not match.")

    def create_backup(self):
        config_path = os.path.join(os.path.dirname(os.path.dirname(__file__)), 'config.json')
        with open(config_path, 'r') as f:
            data = json.load(f)
            if os.path.exists(data['databaseLocation']):
                backup_path = data['databaseLocation'].replace('.db', '-backup.db')
                copyfile(data['databaseLocation'], backup_path)
                QMessageBox.information(self, "Success", "Backup successfully created.")

    def rename_db(self):
        new_name = self.rename_db_input.text()
        if not new_name.endswith('.db'):
            new_name += '.db'
        config_path = os.path.join(os.path.dirname(os.path.dirname(__file__)), 'config.json')
        with open(config_path, 'r+') as f:
            data = json.load(f)
            if os.path.exists(data['databaseLocation']):
                old_name = os.path.basename(data['databaseLocation'])
                new_path = os.path.join(os.path.dirname(data['databaseLocation']), new_name)
                os.rename(data['databaseLocation'], new_path)
                data['databaseLocation'] = new_path
                f.seek(0)
                json.dump(data, f, indent=4)
                f.truncate()
                QMessageBox.information(self, "Success", f"Successfully renamed {old_name} to {new_name}.")

### **<u>logs.py</u>**

The following Python code defines a LogsPage class for a PyQt6 GUI application. The LogsPage class inherits from QWidget and represents a page for viewing and exporting logs from a SQLite database.

The page contains two sections: one for recent reminders and one for admin logs. Each section has a read-only text box that displays the logs and a set of buttons for exporting the logs as .csv, .json, or .xlsx files. The admin logs section is currently commented out.

The load_logs method connects to the SQLite database, executes a SQL SELECT statement to fetch the logs, and populates the reminders text box with the fetched logs. Each log includes the ID, person's name, medication name, action, and timestamp.

The export_data method exports the data from a specified table in the database to a specified file format. It uses the pandas library to read the data from the database and write it to a file. If the database does not exist, it displays a warning message.

The location of the SQLite database is read from a config.json file.

In [ ]:
from PyQt6.QtWidgets import QVBoxLayout, QWidget, QLabel, QPushButton, QTextEdit, QFrame, QHBoxLayout, QMessageBox
from PyQt6.QtGui import QPalette, QColor, QFont
from PyQt6.QtCore import Qt
import os
import json
import sqlite3
import pandas as pd

class LogsPage(QWidget):
    def __init__(self):
        super().__init__()
        with open(os.path.join(os.path.dirname(os.path.dirname(__file__)), 'config.json')) as f:
            self.db_location = json.load(f)['databaseLocation']
        self.layout = QVBoxLayout()

        # Recent reminders section
        self.reminders_label = QLabel("Recent reminders")
        self.reminders_label.setFont(QFont('Arial', 12))
        self.reminders_box = QTextEdit()
        self.reminders_box.setReadOnly(True)
        self.reminders_box.setFrameShape(QFrame.Shape.StyledPanel)
        self.reminders_box.setFrameShadow(QFrame.Shadow.Sunken)
        self.reminders_box.setLineWidth(2)
        self.reminders_box.setMidLineWidth(3)
        self.reminders_box.setContentsMargins(6, 6, 6, 6)  # Set the margins to create the rounded corners
        # Load logs from the database
        self.load_logs()

        # Export buttons for reminders
        self.reminders_export_layout = QHBoxLayout()
        self.reminders_export_csv = QPushButton("Export as .csv")
        self.reminders_export_json = QPushButton("Export as .json")
        self.reminders_export_xlsx = QPushButton("Export as .xlsx")
        self.reminders_export_layout.addWidget(self.reminders_export_csv)
        self.reminders_export_layout.addWidget(self.reminders_export_json)
        self.reminders_export_layout.addWidget(self.reminders_export_xlsx)

        # Connect export buttons to their respective methods
        self.reminders_export_csv.clicked.connect(lambda: self.export_data('reminders', 'csv'))
        self.reminders_export_json.clicked.connect(lambda: self.export_data('reminders', 'json'))
        self.reminders_export_xlsx.clicked.connect(lambda: self.export_data('reminders', 'xlsx'))

        # Add widgets to the main layout
        self.layout.addWidget(self.reminders_label)
        self.layout.addWidget(self.reminders_box)
        self.layout.addLayout(self.reminders_export_layout)

        self.refresh_button = QPushButton("Refresh")
        self.refresh_button.clicked.connect(self.load_logs)
        self.layout.addWidget(self.refresh_button)

        self.setLayout(self.layout)

    def load_logs(self):
        conn = sqlite3.connect(self.db_location)
        cursor = conn.cursor()

        # Check if the log table exists
        cursor.execute("""
            SELECT name FROM sqlite_master WHERE type='table' AND name='log';
        """)
        if cursor.fetchone() is None:
            # The log table does not exist, so return without loading data
            conn.close()
            return

        cursor.execute("""
            SELECT l.id, p.name, m.name, l.action, l.timestamp
            FROM log l
            JOIN reminders r ON l.reminder_id = r.id
            JOIN people p ON r.person_id = p.id
            JOIN medications m ON r.medication_id = m.id
        """)
        logs = cursor.fetchall()

        # Clear the reminders_box before appending new logs
        self.reminders_box.clear()

        for log_id, person, medication, action, timestamp in logs:
            self.reminders_box.append(f"ID: {log_id}, Person: {person}, Medication: {medication}, Action: {action}, Timestamp: {timestamp}")

        conn.close()

    def export_data(self, table_name, file_format):
        config_path = os.path.join(os.path.dirname(os.path.dirname(__file__)), 'config.json')
        with open(config_path, 'r') as f:
            data = json.load(f)
            db_path = data['databaseLocation']
            if os.path.exists(db_path):
                conn = sqlite3.connect(db_path)
                df = pd.read_sql_query(f"SELECT * FROM {table_name}", conn)
                if file_format == 'csv':
                    df.to_csv(f"{table_name}.csv", index=False)
                elif file_format == 'json':
                    df.to_json(f"{table_name}.json", orient='records')
                elif file_format == 'xlsx':
                    df.to_excel(f"{table_name}.xlsx", index=False)
                conn.close()
                QMessageBox.information(self, "Success", f"{table_name} successfully exported as {file_format}.")
            else:
                QMessageBox.warning(self, "Failed", "Database does not exist.")

### **<u>user_settings.py</u>**

The following Python code defines two classes, PasswordLineEdit and UserSettingsPage, for a PyQt6 GUI application.

The PasswordLineEdit class inherits from QLineEdit and represents a password input field. It has an action with an eye icon that toggles the visibility of the password. The action is visible only when the input field is focused.

The UserSettingsPage class inherits from QWidget and represents a user settings page. The page contains two forms for changing the user's email and password. Each form has three input fields (current, new, and confirm) and a button. The password input fields are instances of PasswordLineEdit.

The UserSettingsPage class does not currently implement any functionality for the buttons.

In [ ]:
from PyQt6.QtWidgets import QVBoxLayout, QWidget, QLabel, QLineEdit, QPushButton
from PyQt6.QtGui import QAction
from qtawesome import icon

class PasswordLineEdit(QLineEdit):
    def __init__(self, parent=None):
        super().__init__(parent)
        self.reveal_password_action = QAction(icon('fa5s.eye'), 'Reveal Password')
        self.reveal_password_action.triggered.connect(self.toggle_password_visibility)
        self.addAction(self.reveal_password_action, QLineEdit.ActionPosition.TrailingPosition)
        self.setEchoMode(QLineEdit.EchoMode.Password)
        self.reveal_password_action.setVisible(False)

    def focusInEvent(self, event):
        super().focusInEvent(event)
        self.reveal_password_action.setVisible(True)

    def focusOutEvent(self, event):
        super().focusOutEvent(event)
        self.reveal_password_action.setVisible(False)
        self.setEchoMode(QLineEdit.EchoMode.Password)

    def toggle_password_visibility(self):
        if self.echoMode() == QLineEdit.EchoMode.Password:
            self.setEchoMode(QLineEdit.EchoMode.Normal)
        else:
            self.setEchoMode(QLineEdit.EchoMode.Password)

class UserSettingsPage(QWidget):
    def __init__(self):
        super().__init__()
        self.layout = QVBoxLayout()
        self.email_label = QLabel("Email")
        self.current_email = QLineEdit()
        self.new_email = QLineEdit()
        self.confirm_email = QLineEdit()
        self.change_email_btn = QPushButton("Change Email")
        self.password_label = QLabel("Password")
        self.current_password = PasswordLineEdit()
        self.new_password = PasswordLineEdit()
        self.confirm_password = PasswordLineEdit()
        self.change_password_btn = QPushButton("Change Password")
        self.layout.addWidget(self.email_label)
        self.layout.addWidget(self.current_email)
        self.layout.addWidget(self.new_email)
        self.layout.addWidget(self.confirm_email)
        self.layout.addWidget(self.change_email_btn)
        self.layout.addWidget(self.password_label)
        self.layout.addWidget(self.current_password)
        self.layout.addWidget(self.new_password)
        self.layout.addWidget(self.confirm_password)
        self.layout.addWidget(self.change_password_btn)
        self.setLayout(self.layout)

### **<u>What Could Improve / Isn't Finished</u>**